# Training Tensorflow Objecte Detection

The most convenient way to train a TensorFlow object detection model is to use verified Tensorflow models architectures provided by TensorFlow. you can find the GitHub repo at this link [TensorFlow official](https://github.com/tensorflow/models).

In this section, I train an object detection model (EfficientDet D3) in a virtual environment. The reason for using a virtual environment is to make the whole process separate so that it won't affect other model implements or environment setups.

I use an anaconda virtual environment to train this model. You can use any kind of virtual environment setup to train the model. the benefit of using an anaconda environment is, when I install TensorFlow GPU using conda commands, it automatically install compatible Cuda and Cudnn libraries. if you use other environments, you may or may not have to set up these two necessary libraries.

### Install anaconda virtual environment

installing anaconda is fairly easy. Open a terminal copy-paste the following commands.

<pre>
sudo apt install libgl1-mesa-glx libegl1-mesa libxrandr2 libxrandr2 libxss1 libxcursor1 libxcomposite1 libasound2 libxi6 libxtst6

wget -P /tmp https://repo.anaconda.com/archive/Anaconda3-2020.02-Linux-x86_64.sh

bash /tmp/Anaconda3-2020.02-Linux-x86_64.sh

source ~/.bashrc
</pre>

This process will ask for license approval. just approve the license and you are ready to go.

After installing an anaconda-navigator, close the terminal and open a new one. Now create a new anaconda virtual environment. Simple type -
<pre>conda create -n tensorflow pip python=3.6</pre>
where "tensorflow" is the name of your environment. You can choose any python version. I use python 3.6 as I know this version is one of the most stable versions.

After creating the virtual environment activate the environment. Simply type **source activate tensorflow**. The terminal will show you this command once the environment installation is successfully finished.

Next, install TensorFlow-GPU (as we will use GPU to train) on our system. simply type -

<pre>conda install -c anaconda tensorflow-gpu</pre>

This will install the TensorFlow-GPU version to our system as well as compatible Cuda and Cudnn in this virtual environment.

That's all for an anaconda virtual environment setup.<br>
**Note: all these commands should be run in a terminal, not in the Jupyter notebook.**

## Install TensorFlow Object Detection API

Now open a jupyter notebook for the next installation process that is TensorFlow Object Detection API. Simply type -
<pre> jupyter notebook</pre> to open a jupyter notebook.

Now you may face some errors as jupyter notebook is not always installed with the new environment setup. If this happens for you simply type - <pre> pip install notebook </pre> This will install a new jupyter notebook for this virtual environment.<br>

Now open jupyter notebook.

First check the installed version of Tensorflow. I installed Tensorflow 2.4.1. Your version might be different.

In [1]:
import tensorflow as tf
tf.__version__

2022-01-26 13:49:37.407858: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


'2.4.1'

### Downloading the TensorFlow Model Garden

First create a folder where you like to store all your files(pre-trained model, trained model, data, etc) related to object detection. then go to that directory and execute the following command.

In [3]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow'

In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 68544, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 68544 (delta 2), reused 9 (delta 2), pack-reused 68534
Receiving objects: 100% (68544/68544), 576.95 MiB | 14.26 MiB/s, done.
Resolving deltas: 100% (48277/48277), done.


This will download Tensorflow model garden in your folder. I named my model as **"TensorFlow"**. You can name anything you want.

### Installat Protobuf 

Tensorflow Object Detection API uses Protobufs to configure model and training parameters. This library must be installed inside the **"research"** folder, which is inside the **"models"** folder. BTW, the **"models"** folder is the Tensorflow model garden repo which we pulled earlier.

In [4]:
cd models/research/

/home/ubuntu-20/Desktop/TensorFlow/models/research


In [5]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/models/research'

Now execute the Protobuf installation command here.

In [6]:
!protoc object_detection/protos/*.proto --python_out=.

### Install COCO API

"pycocotools" is another dependency needed for Tensorflow object detection API. You can simply pull the library from GitHub and install/make it. But before doing that, you need "cython" library to be installed. Install "cython" as follows -

In [10]:
!pip install cython

     |████████████████████████████████| 1.9 MB 7.6 MB/s eta 0:00:01


Next clone "pycocotools" repo from GitHub and execute the following commands one by one.

In [7]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 13.58 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [8]:
cd cocoapi/PythonAPI

/home/ubuntu-20/Desktop/TensorFlow/models/research/cocoapi/PythonAPI


In [11]:
!make

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/home/ubuntu-20/anaconda3/envs/tensorflow/lib/python3.9/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/ubuntu-20/Desktop/TensorFlow/models/research/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
gcc -pthread -B /home/ubuntu-20/anaconda3/envs/tensorflow/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/ubuntu-20/anaconda3/envs/tensorflow/include -I/home/ubuntu-20/anaconda3/envs/tensorflow/include -fPIC -O2 -isystem /home/ubuntu-20/anaconda3/envs/tensorflow/include -fPIC -I/home/ubuntu-20/anaconda3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/include -I../common -I/home/ubuntu-20/anaconda3/envs/tensorflow/include/python3.

In [12]:
cp -r pycocotools /home/ubuntu-20/Desktop/TensorFlow/models/research

So,"pycocotools" library is installed successfully.

### Install the Object Detection API

Now all the dependencies necessary for installing Object Detection API are done. So, we can install Object Detection API now. Object Detection API should be installed inside the **"research"** directory. Check the present current directory and go to the **"research"** directory

In [13]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/models/research/cocoapi/PythonAPI'

In [14]:
cd ..

/home/ubuntu-20/Desktop/TensorFlow/models/research/cocoapi


In [15]:
cd ..

/home/ubuntu-20/Desktop/TensorFlow/models/research


In [16]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/models/research'

Next run the following two commands to install **"Object Detection API"**

In [17]:
cp object_detection/packages/tf2/setup.py .

In [18]:
!python -m pip install .

Processing /home/ubuntu-20/Desktop/TensorFlow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 2.6 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 27.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 20.1 MB/s eta 0:00:01
     |████████████████████████████████| 11.2 MB 31.2 MB/s eta 0:00:01
     |████████████████████████████████| 352 kB 31.6 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 54.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
 

     |████████████████████████████████| 76 kB 3.8 MB/s  eta 0:00:01
     |████████████████████████████████| 503 kB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.3 MB 37.6 MB/s eta 0:00:01
     |████████████████████████████████| 463 kB 36.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 152 kB 29.7 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 89 kB 7.4 MB/s  eta 0:00:01
     |████████████████████████████████| 245 kB 69.9 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 33.1 MB/s eta 0:00:01


     |████████████████████████████████| 2.5 MB 22.3 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 516 kB 40.7 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 3.0 MB/s  eta 0:00:01
     |████████████████████████████████| 25.6 MB 23.7 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 4.9 MB/s  eta 0:00:01
     |████████████████████████████████| 1.6 MB 48.6 MB/s eta 0:00:01
  Using cached opencv_python-4.5.5.62-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.4 MB)
     |████████████████████████████████| 895 kB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 22.5 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 6.7 MB/s  eta 0:00:011
     |████████████████████████████████| 763 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 26.4 MB 29.3 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 31.9 MB/s eta 0:00:0

  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.21.3
    Uninstalling google-auth-1.21.3:
      Successfully uninstalled google-auth-1.21.3
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.7
    Uninstalling pyparsing-3.0.7:
      Successfully uninstalled pyparsing-3.0.7
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.0.1
    Uninstalling typing-extensions-4.0.1:
      Successfully uninstalled typing-extensions-4.0.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.1
    Uninstalling tens

Now object detection setup is done. You can verify the installation by executing the following command.

In [19]:
!python object_detection/builders/model_builder_tf2_test.py

2022-01-26 14:06:57.624502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 14:06:57.627064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2022-01-26 14:06:57.627577: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Running tests under Python 3.9.7: /home/ubuntu-20/anaconda3/envs/tensorflow/bin/python
[ RUN      ] ModelBuilderTF2T

I0126 14:07:01.496387 139775303639424 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet EfficientNet backbone version: efficientnet-b0
I0126 14:07:01.496513 139775303639424 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num filters: 64
I0126 14:07:01.496586 139775303639424 ssd_efficientnet_bifpn_feature_extractor.py:148] EfficientDet BiFPN num iterations: 3
I0126 14:07:01.498541 139775303639424 efficientnet_model.py:147] round_filter input=32 output=32
I0126 14:07:01.511177 139775303639424 efficientnet_model.py:147] round_filter input=32 output=32
I0126 14:07:01.511280 139775303639424 efficientnet_model.py:147] round_filter input=16 output=16
I0126 14:07:01.567244 139775303639424 efficientnet_model.py:147] round_filter input=16 output=16
I0126 14:07:01.567365 139775303639424 efficientnet_model.py:147] round_filter input=24 output=24
I0126 14:07:01.709391 139775303639424 efficientnet_model.py:147] round_filter input=24 output=24
I0126 14:07:01.709509 139

I0126 14:07:04.986997 139775303639424 efficientnet_model.py:147] round_filter input=16 output=16
I0126 14:07:04.987106 139775303639424 efficientnet_model.py:147] round_filter input=24 output=24
I0126 14:07:05.199536 139775303639424 efficientnet_model.py:147] round_filter input=24 output=24
I0126 14:07:05.199649 139775303639424 efficientnet_model.py:147] round_filter input=40 output=48
I0126 14:07:05.415032 139775303639424 efficientnet_model.py:147] round_filter input=40 output=48
I0126 14:07:05.415156 139775303639424 efficientnet_model.py:147] round_filter input=80 output=88
I0126 14:07:05.802588 139775303639424 efficientnet_model.py:147] round_filter input=80 output=88
I0126 14:07:05.802737 139775303639424 efficientnet_model.py:147] round_filter input=112 output=120
I0126 14:07:06.088043 139775303639424 efficientnet_model.py:147] round_filter input=112 output=120
I0126 14:07:06.088167 139775303639424 efficientnet_model.py:147] round_filter input=192 output=208
I0126 14:07:06.498754 13

I0126 14:07:11.172519 139775303639424 efficientnet_model.py:147] round_filter input=112 output=160
I0126 14:07:11.172716 139775303639424 efficientnet_model.py:147] round_filter input=192 output=272
I0126 14:07:11.924983 139775303639424 efficientnet_model.py:147] round_filter input=192 output=272
I0126 14:07:11.925154 139775303639424 efficientnet_model.py:147] round_filter input=320 output=448
I0126 14:07:12.110312 139775303639424 efficientnet_model.py:147] round_filter input=1280 output=1792
I0126 14:07:12.156013 139775303639424 efficientnet_model.py:457] Building model efficientnet with params ModelConfig(width_coefficient=1.4, depth_coefficient=1.8, resolution=380, dropout_rate=0.4, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_rati

I0126 14:07:20.166972 139775303639424 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet EfficientNet backbone version: efficientnet-b7
I0126 14:07:20.167091 139775303639424 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num filters: 384
I0126 14:07:20.167124 139775303639424 ssd_efficientnet_bifpn_feature_extractor.py:148] EfficientDet BiFPN num iterations: 8
I0126 14:07:20.168422 139775303639424 efficientnet_model.py:147] round_filter input=32 output=64
I0126 14:07:20.185153 139775303639424 efficientnet_model.py:147] round_filter input=32 output=64
I0126 14:07:20.185356 139775303639424 efficientnet_model.py:147] round_filter input=16 output=32
I0126 14:07:20.498679 139775303639424 efficientnet_model.py:147] round_filter input=16 output=32
I0126 14:07:20.498799 139775303639424 efficientnet_model.py:147] round_filter input=24 output=48
I0126 14:07:21.150535 139775303639424 efficientnet_model.py:147] round_filter input=24 output=48
I0126 14:07:21.150656 13

## Train Object Detection

To train an Object Detection model, we need some assets like some images for both train and test, image annotations, pre-trained object detection model, trained resources, and exported model to use for prediction. To keep things clear I create some directories. All the new folders are created inside the named **"training_demo"**.

In [20]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/models/research'

**"training_demo"** for organizing all the resources

In [21]:
!mkdir ../../training_demo

**"annotations"** for ".record" and ".pbtxt" files. ".record" files are used as annotations and ".pbtxt" file for class names.

In [22]:
!mkdir ../../training_demo/annotations

**"exported-models"** for final exported models. This model (.pb file and .config file is used for final prediction)

In [23]:
!mkdir ../../training_demo/exported-models

**"images"** folder for store train and validation images

In [24]:
!mkdir ../../training_demo/images

In [27]:
!mkdir ../../training_demo/images/train

In [28]:
!mkdir ../../training_demo/images/test

**"models"** directory to store models and training informations. 

In [25]:
!mkdir ../../training_demo/models

**"pre-trained-models"** for storing "pre-trained-models" models. in our case "efficientdetD3"

In [26]:
!mkdir ../../training_demo/pre-trained-models

### Download pre-trained model

In [29]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/models/research'

change directory to "pre-trained-models" (which I create earlier). Then downlaod pre-train model from tensorflow model zoo. To download any model go to this [GitHub repo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Copy any model's link address and download it using wget

In [30]:
cd ../../training_demo/pre-trained-models

/home/ubuntu-20/Desktop/TensorFlow/training_demo/pre-trained-models


In [31]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/training_demo/pre-trained-models'

In [32]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d3_coco17_tpu-32.tar.gz

--2022-01-26 16:12:25--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d3_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 2404:6800:4004:810::2010, 172.217.174.112
Connecting to download.tensorflow.org (download.tensorflow.org)|2404:6800:4004:810::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92858658 (89M) [application/x-tar]
Saving to: ‘efficientdet_d3_coco17_tpu-32.tar.gz’

efficientdet_d3_coc 100%[===================>]  88.56M  27.6MB/s    in 3.5s    

2022-01-26 16:12:29 (25.6 MB/s) - ‘efficientdet_d3_coco17_tpu-32.tar.gz’ saved [92858658/92858658]



unpack downloaded model inside "pre-trained-models"

In [33]:
!tar -xvf efficientdet_d3_coco17_tpu-32.tar.gz

efficientdet_d3_coco17_tpu-32/
efficientdet_d3_coco17_tpu-32/checkpoint/
efficientdet_d3_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
efficientdet_d3_coco17_tpu-32/checkpoint/checkpoint
efficientdet_d3_coco17_tpu-32/checkpoint/ckpt-0.index
efficientdet_d3_coco17_tpu-32/pipeline.config
efficientdet_d3_coco17_tpu-32/saved_model/
efficientdet_d3_coco17_tpu-32/saved_model/saved_model.pb
efficientdet_d3_coco17_tpu-32/saved_model/assets/
efficientdet_d3_coco17_tpu-32/saved_model/variables/
efficientdet_d3_coco17_tpu-32/saved_model/variables/variables.data-00000-of-00001
efficientdet_d3_coco17_tpu-32/saved_model/variables/variables.index


In [34]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/training_demo/pre-trained-models'

### Generate .record file

Now we have a pre-trained model. Let's create training and test files. We have some training images along with annotation XML files both in the train and test folder. There is a **"generate_tfrecord.py"** file inside the "training_demo" folder. This file can be used to generate train and test the ".record" file.<br>

simply call "generate_tfrecord.py" and pass train images folder path, label_map.pbtxt file path, and directory path to save train.record file path. Paths can be anywhere. it completely depends on you.

In [35]:
cd ..

/home/ubuntu-20/Desktop/TensorFlow/training_demo


In [50]:
# Create train data:
!python generate_tfrecord.py -x images/train -l annotations/label_map.pbtxt -o annotations/train.record

#Create test data:
!python generate_tfrecord.py -x images/test -l annotations/label_map.pbtxt -o annotations/test.record

Successfully created the TFRecord file: annotations/train.record
Successfully created the TFRecord file: annotations/test.record


Now we have necessary training files (train.record, test.record and label_map.pbtxt)and necessary pre-traind models. One last thing is needed is to configer **"pipeline.config"** file. Inside this file we have to configer

- train.record file path
- test.record file path
- label_map.pbtxt file path
- batch_size
- number of classes
- fine_tune_checkpoint_type
- use_bfloat16
- fine_tune_checkpoint_type

You can find "pipeline.config" file inside the downloaded pre-trained model folder

To make everything clean and reusable, I coppied the "pipeline.config" file from "pre-trained-models/efficientdet_d3_coco17_tpu-32" to "exported_models/efficientdet" folder and edit it. Open "pipeline.config" and searche for 
1. num_classes
<pre>As we are focusing just on finding the object, our number of classes is 1. Now change num_classes to 1.</pre>
2. batch_size
<pre>Depending of the GPU, we can use any batch size. So use any number suitable for the training</pre>
3. total_steps/num_steps
<pre>Defines how long we like to train the model. Note that total_steps & num_steps are two different parameters. You have to assign the same value for both of them. Another important point is "warmup_steps". Both total_steps & num_steps must be higher than warmup_steps. Otherwise, you will get some errors.</pre>
4. fine_tune_checkpoint
<pre>You may find it just before the "num_steps" parameter. Basically, you have to specify the downloaded pre-trained model's checkpoint path to start training using that pre-trained model. Currently, we are using efficientdetD3. So, go to the pre-trained-models -> efficientdet_d3_coco17_tpu-32 -> checkpoint folder  and copy the relative path of "ckpt-0.index" and paste it here. As I am currently inside "training_demo" folder, so relative path from "training_demo" would be "./pre-trained-models/efficientdet_d3_coco17_tpu-32/checkpoint/ckpt-0". <strong>Note that, you have to remove the extension from "ckpt-0.index" file</strong>. The model already knows the file extension. <strong>BTW, all folder names are changeable. If you want to change the folder name or path, you can do that. Just make sure to specify the file path correctly</strong></pre>
5. fine_tune_checkpoint_type
<pre>Change fine_tune_checkpoint_type to <strong>"detection"</strong> as we are trying to detect objects.</pre>
6. use_bfloat16
<pre>Change "use_bfloat16" to <strong>flase</strong> as we are not using TPU's</pre>
7. train_input_reader
<pre>Inside "train_input_reader" section, you have to modify 2 parameters, "label_map_path" & "input_path".  The "label_map.pbtxt" can be found inside "annotations" folder (as i put it there. You can put it anywhere you like). As we are still inside "training_demo" folder, so relative path would be "./annotations/label_map.pbtxt".<br>
Next "input_path". As this "input_path" is inside "train_input_reader" section, so we have to specify traing input file which is <strong>train.record</strong> file. The "train.record" file can also be found inside "annotations" folder. As we are still inside "training_demo" folder, so relative path would be "./annotations/train.record".</pre>

8. eval_input_reader
<pre>Inside "eval_input_reader" section, you also have to modify 2 parameters, "label_map_path" & "input_path".  The "label_map.pbtxt" can be found inside "annotations" folder (as i put it there. You can put it anywhere you like). As we are still inside "training_demo" folder, so relative path would be "./annotations/label_map.pbtxt".<br>
Next "input_path". As this "input_path" is inside the "eval_input_reader" section, so we have to specify the training input file which is <strong>test.record</strong> file (I named validation set as test. You can use val or validation. Just make sure to change the path accordingly). The "test.record" file can also be found inside the "annotations" folder. As we are still inside the "training_demo" folder, so the relative path would be "./annotations/test.record".</pre>

### Training

Now we are done with all installation and configurations. Let's train the model. Currently we are inside **"training_demo"** folder. If you check, you will find a file named **"model_main_tf2.py"**. This is the file we are going to use to train the object detection model. This file is also provided by Tensorflow. Check out this file. There are also some configurations inside this file. You can change/pass the argument depending on your objective.

In [51]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/training_demo'

In [52]:
ls

annotations/         export_tflite_graph_tf2.py  model_main_tf2.py
exported-models/     generate_tfrecord.py        models/
exporter_main_v2.py  images/                     pre-trained-models/


To train the model, you have to pass two parameters, **"model_dir"** & **"pipeline_config_path"**. "model_dir" is the folder where you want to save your model training progress (this folder will be needed later for visualization of the training graphs) and "pipeline_config_path" is the folder where we want to save our trained model configuration. (Basically, the configuration file which will be saved in this training process is the same one we configured earlier. Just to make this separate, I saved this config file in the same folder where the trained models are going to save). 

In [62]:
!python model_main_tf2.py --model_dir=models/EfficientDet-D3-custom-trained --pipeline_config_path=models/EfficientDet-D3-custom-trained/pipeline.config

2022-01-26 19:18:16.924165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 19:18:16.927108: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu-20/anaconda3/envs/tensorflow/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64
2022-01-26 19:18:16.927694: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-26 19:18:16.928442: I 

Instructions for updating:
Use `tf.data.Dataset.map()
W0126 19:18:19.851195 139684560966016 deprecation.py:341] From /home/ubuntu-20/anaconda3/envs/tensorflow/lib/python3.9/site-packages/object_detection/builders/dataset_builder.py:236: DatasetV1.map_with_legacy_function (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.map()
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0126 19:18:24.580934 139684560966016 deprecation.py:341] From /home/ubuntu-20/anaconda3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1096: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use `tf.cast` instead.
W0126 19:18:27.232981 139684560966016 de

If you to train on multiple GPU, then you have to add **"num_clones"**. "num_clones" defines how many GPUs you like to use. I have only 1 GPU. So i assign **--num_clones=1**

In [66]:
!python model_main_tf2.py  --model_dir=models/EfficientDet-D3-custom-trained --pipeline_config_path=models/EfficientDet-D3-custom-trained/pipeline.config --num_clones=1 --ps_tasks=1

2022-01-26 19:51:11.329579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 19:51:11.332355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ubuntu-20/anaconda3/envs/tensorflow/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64
2022-01-26 19:51:11.332701: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-26 19:51:11.333297: I 

Instructions for updating:
Use `tf.data.Dataset.map()
W0126 19:51:14.562267 140671546012032 deprecation.py:341] From /home/ubuntu-20/anaconda3/envs/tensorflow/lib/python3.9/site-packages/object_detection/builders/dataset_builder.py:236: DatasetV1.map_with_legacy_function (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.map()
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0126 19:51:19.616015 140671546012032 deprecation.py:341] From /home/ubuntu-20/anaconda3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1096: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use `tf.cast` instead.
W0126 19:51:22.496867 140671546012032 de

In [61]:
pwd

'/home/ubuntu-20/Desktop/TensorFlow/training_demo'

# Visualizing the training process

Tensorflow has a nice tool to visulize the praining progress named "Tensorbord". This tool is automatically installed when you install tensorflow using the conda comand. To visulize training progress: <br>

- Open a new terminal.
- activate the virtual environment by trping **source activate tensorflow** (which we have done before in a different terminal)
- navigate to **"training_demo"** folder
- now type **tensorboard --logdir=models/EfficientDet-D3-custom-trained** as we saved our trained network in "EfficientDet-D3-custom-trained" folder.

Once this is done, terminal will show you something like this **"TensorBoard 2.6.0 at http://localhost:6006/ (Press CTRL+C to quit)"**<br>

Go to **http://localhost:6006/** and you can see different graphs related to your training progress.(Port number might be different)

# Export the trained model

We already trained our model. But Tensorflow Object Detection API didn't save the final model. Instead, it saves the checkpoints after every 1000 steps(you can change this step number from model_main_tf2.py). Now we have to export the final model from these checkpoints. <br> <br>
**Note:** the benefit of having these checkpoint files is, you can restart your training process from any of these checkpoint files later if you want.<br><br>
Now to export the final model, we will use "exporter_main_v2.py". This file is also provided by Tensorflow.<br><br>

Execute the following command with appropriate parameters. We already discussed all the folders necessary for this command.

In [65]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/EfficientDet-D3-custom-trained/pipeline.config --trained_checkpoint_dir models/EfficientDet-D3-custom-trained --output_directory exported_models/efficientdet

2022-01-26 19:38:08.131745: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 19:38:08.134366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2022-01-26 19:38:08.134966: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-26 19:38:08.140429: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized